In [ ]:
def loan_api_tool(customer_data: Dict) -> Dict:
    """Call external loan API for decision"""
    try:
        response = requests.post(LOAN_API_URL, json=customer_data, timeout=10)
        print(response)
        return response.json()
    except Exception as e:
        return {"decision": "rejected", "reason": f"API call failed: {str(e)}"}

In [ ]:
def loan_agent_node(state: AgentState) -> AgentState:
    """Handle loan decision requests"""
    try:
        # Extract loan request data from database
        cypher_result = cypher_generation_tool("MATCH (c:Customer)-[:LINKED_WITH]->(a:Account) RETURN c.name as name, c.phone as phone, c.email as email, a.balance as balance")
        
        if cypher_result["success"] and cypher_result["result"]:
            customer_data = cypher_result["result"][0]  # Take first customer for demo
            
            # Call loan API
            api_response = loan_api_tool(customer_data)
            
            # Generate loan report
            report_prompt = prompts_dict["loan_report"]["template"].format(
                customer_data=customer_data,
                loan_decision=api_response
            )
            
            response = llm.invoke(report_prompt)
            final_response = response.content
        else:
            final_response = "Error: Could not retrieve customer data for loan analysis"
            
    except Exception as e:
        final_response = f"Loan processing failed: {str(e)}"
    
    return {**state, "final_response": final_response}